In [ ]:
from twitter import *
from __future__ import unicode_literals

In [ ]:
consumer_key=''
consumer_secret = ''
token = ''
token_secret = ''
auth = OAuth(token, token_secret, consumer_key, consumer_secret)
t = Twitter(
    auth=auth)

In [ ]:
t.statuses.home_timeline()

In [ ]:
r=t.search.tweets(q='#cyberthreat')

In [ ]:
len(r)

In [ ]:
r['statuses'][0].keys()

In [ ]:
[ (tweet['created_at'],tweet['text']) for tweet in r['statuses']]

In [ ]:
r['statuses'][0].keys()

In [ ]:
r['statuses'][0]['text']

In [ ]:
twitter_userstream = TwitterStream(auth=auth, domain='userstream.twitter.com')
hash_tag ='JourneeDeLaGentillesse'

In [ ]:
for tweet in twitter_userstream.statuses.filter(track=hash_tag):
    break

In [ ]:
tweet['text']

In [ ]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer

In [ ]:
schema = Schema(ID_Tweet=ID(stored=True),
                user=TEXT(stored=True),
                body=TEXT(analyzer=StemmingAnalyzer(),stored=True),
                tags=KEYWORD)

In [ ]:
schema

In [ ]:
import os, os.path
from whoosh import index

In [ ]:
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

In [ ]:
import whoosh.index as index
from whoosh.filedb.filestore import FileStorage


In [ ]:
storage = FileStorage("indexdir")

In [ ]:
ix = storage.create_index(schema)

In [ ]:
ix

In [ ]:
writer = ix.writer()

In [ ]:
tweet.keys()

In [ ]:
_id = str(tweet['id'])
user = tweet['user']['name']
content = tweet['text']
type(_id)

In [ ]:
writer.add_document(ID_Tweet=_id,body=content,user=user,tags='#cyberthreat')

In [ ]:
tweet['text']

In [ ]:
tweet['user']['name']

In [ ]:
#writer.commit()

In [ ]:
from whoosh.qparser import QueryParser

In [ ]:
qp = QueryParser("body", schema=ix.schema)
q = qp.parse("RT")

with ix.searcher() as s:
    results = s.search(q)
    for r in results:
        print(r.items())

In [ ]:
i=0
limit=20
hash_tag='#cyberthreat'
twitter_userstream = TwitterStream(auth=auth, domain='userstream.twitter.com')

for tweet in twitter_userstream.statuses.filter(track=hash_tag):
    if i == limit:
        writer.commit()
        break
    _id = str(tweet['id'])
    user = tweet['user']['name']
    content = tweet['text']
    writer.add_document(ID_Tweet=_id,body=content,user=user,tags='#cyberthreat')
    i+=1
    print('%s %s'% (i,content) )

In [ ]:
writer.commit()

In [ ]:
qp = QueryParser("body", schema=ix.schema)
q = qp.parse("RT")
i=0
with ix.searcher() as s:
    results = s.search(q,limit=None)
    print(len(results))
    for r in results:
        print(i)
        print(r.items())
        i+=1

In [ ]:
all_docs = ix.searcher().documents() 

In [ ]:
i = 0
for doc in all_docs:
    print(doc['body'])
    print(i)
    i+=1